## Get all Symbol Exchange

In [2]:
import ccxt
import pandas as pd


def get_exchange_symbols(exchange_id):
    """
    Fetch all trading symbols for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of trading pair symbols (e.g., ["BTC/USDT", "ETH/USDT"]).
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        
        # Load markets
        markets = exchange.load_markets()
        
        # Extract symbols
        symbols = list(markets.keys())
        return symbols

    except Exception as e:
        print(f"Error fetching symbols for exchange {exchange_id}: {e}")
        return []

In [3]:

exchange_id = "binance"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

exchange_id = "bitget"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

Total symbols on binance: 3339
Sample symbols: ['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH', 'EOS/ETH', 'SNT/ETH', 'BNT/ETH', 'BCC/BTC', 'GAS/BTC']
Total symbols on bitget: 1238
Sample symbols: ['FTM/EUR', 'CATCH/USDT', 'LUMIA/USDT', 'GOAT/USDT', 'ENA/EUR', 'TON/EUR', 'SXRP/SUSDT', 'SHIB/EUR', 'WLD/EUR', 'DBR/USDT']


In [4]:
def get_top_usdt_symbol_by_volume(exchange_id, top_n=100):
    """
    Fetch the top N cryptocurrencies traded against USDT based on 24h volume.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").
        top_n (int): The number of top cryptocurrencies to fetch (default is 100).

    Returns:
        pd.DataFrame: A DataFrame containing the top N USDT pairs by 24h volume.
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        
        # Fetch all tickers
        tickers = exchange.fetch_tickers()
        
        # Extract relevant data for USDT pairs
        data = []
        for symbol, ticker in tickers.items():
            if symbol.endswith("/USDT"):  # Filter for USDT pairs
                data.append({
                    "symbol": symbol,
                    "volume_24h": ticker.get("quoteVolume", 0),  # 24h trading volume in USDT
                    "price": ticker.get("last", 0),  # Last traded price
                })
        
        # Convert to DataFrame and sort by volume
        df = pd.DataFrame(data)
        df = df.sort_values(by="volume_24h", ascending=False).head(top_n)
        
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [5]:
exchange_id = "binance"  # Replace with your desired exchange ID
top_symbols = get_top_usdt_symbol_by_volume(exchange_id, top_n=100)
binance_symbols = pd.DataFrame(top_symbols)
binance_symbols = binance_symbols.rename(columns={"volume_24h": f"volume_24h_{exchange_id}"})
binance_symbols = binance_symbols.rename(columns={"price": f"price_{exchange_id}"})
binance_symbols = binance_symbols.reset_index(drop=True)
binance_symbols.to_csv(f"/home/ubuntu/project/finance/cex-market-analysis/symbols/top_100_{exchange_id}.csv")


exchange_id = "bitget"  # Replace with your desired exchange ID
top_symbols = get_top_usdt_symbol_by_volume(exchange_id, top_n=100)
bitget_symbols = pd.DataFrame(top_symbols)
bitget_symbols = bitget_symbols.rename(columns={"volume_24h": f"volume_24h_{exchange_id}"})
bitget_symbols = bitget_symbols.rename(columns={"price": f"price_{exchange_id}"})
bitget_symbols = bitget_symbols.reset_index(drop=True)
bitget_symbols.to_csv(f"/home/ubuntu/project/finance/cex-market-analysis/symbols/{exchange_id}.csv")
df = pd.merge(binance_symbols, bitget_symbols, on=['symbol'])
df

,symbol,volume_24h_binance,price_binance,volume_24h_bitget,price_bitget
0,BTC/USDT,1.955964e+09,97439.270000,6.616714e+08,97408.000000
1,ETH/USDT,1.135309e+09,3472.560000,5.121737e+08,3473.000000
2,XRP/USDT,1.069669e+09,2.425900,2.369296e+08,2.426300
3,USDC/USDT,9.411158e+08,1.000800,2.496265e+07,1.000700
4,SOL/USDT,6.534331e+08,207.570000,1.008028e+08,207.570000
5,DOGE/USDT,5.517312e+08,0.339650,1.145052e+08,0.339750
6,PEPE/USDT,4.460193e+08,0.000021,1.979347e+07,0.000021
7,XLM/USDT,3.338199e+08,0.439800,5.449653e+07,0.439600
8,ADA/USDT,2.422317e+08,0.961200,4.621460e+07,0.961200
9,BNB/USDT,2.009081e+08,708.490000,4.998481e+06,708.700000


In [45]:
def get_exchange_timeframes(exchange_id):
    """
    Get the list of available timeframes for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of available timeframes for the exchange.
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})

        # Check if the exchange supports timeframes
        if hasattr(exchange, 'timeframes') and exchange.timeframes:
            return list(exchange.timeframes.keys())
        else:
            return []
    except Exception as e:
        print(f"Error fetching timeframes for {exchange_id}: {e}")
        return []

# Example Usage
exchange_id = "bitget"  # Replace with your desired exchange ID
timeframes = get_exchange_timeframes(exchange_id)
print(f"Available timeframes for {exchange_id}: {timeframes}")

Available timeframes for bitget: ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '12h', '1d', '3d', '1w', '1M']


In [9]:
import sys
from pathlib import Path

# Add the parent directory to the system path
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))

# Now you can import your module
from utils.general import get_top_usdt_symbol_by_volume

# Example usage
exchange_id = "binance"
top_symbols = get_top_usdt_symbol_by_volume(exchange_id, top_n=100)


In [11]:
top_symbols["symbol"]

0           BTC/USDT
1           ETH/USDT
8           XRP/USDT
23         USDC/USDT
119         SOL/USDT
           ...      
471    1000SATS/USDT
476         XAI/USDT
513         POL/USDT
509         TON/USDT
479         JUP/USDT
Name: symbol, Length: 100, dtype: object